# Bubble Chart spätester Sonnenaufgang

In [13]:
import geopandas as gpd
import pandas as pd
import json
import consts
from utils import categorize_time_to_half_hour, categorize_time_to_half_hour_numerical, dt_to_time, time_to_time, categorize_time_to_6min_numerical, categorize_time_to_6min
import pyperclip
from pathlib import Path
import datetime

In [2]:
# Load timedata
time_data = json.load(open(consts.PATH_TIMEDATA_HEX, 'r', encoding='utf-8'))

# Load Nuts3
gdf_grid_raw = gpd.read_file(consts.PATH_HEXAGON)

## Extremwerte pro Land

In [ ]:


properties

['sunrise_earliest_current',
 'sunrise_latest_current',
 'sunset_earliest_current',
 'sunset_latest_current',
 'sunrise_earliest_winter',
 'sunrise_latest_winter',
 'sunset_earliest_winter',
 'sunset_latest_winter',
 'sunrise_earliest_summer',
 'sunrise_latest_summer',
 'sunset_earliest_summer',
 'sunset_latest_summer']

In [27]:
# Serialize Timedata
df_timedata = list(map(lambda x: {
    'nuts_id': x['nuts_id'],

    'sunrise_earliest_current': x['sunrise']['current']['earliest'],
    'sunrise_latest_current': x['sunrise']['current']['latest'],
    'sunset_earliest_current': x['sunset']['current']['earliest'],
    'sunset_latest_current': x['sunset']['current']['latest'],

    'sunrise_earliest_winter': x['sunrise']['winter']['earliest'],
    'sunrise_latest_winter': x['sunrise']['winter']['latest'],
    'sunset_earliest_winter': x['sunset']['winter']['earliest'],
    'sunset_latest_winter': x['sunset']['winter']['latest'],

    'sunrise_earliest_summer': x['sunrise']['summer']['earliest'],
    'sunrise_latest_summer': x['sunrise']['summer']['latest'],
    'sunset_earliest_summer': x['sunset']['summer']['earliest'],
    'sunset_latest_summer': x['sunset']['summer']['latest'],

}, time_data))

df_timedata = pd.DataFrame(df_timedata)

def apply_time(dt):
    return datetime.datetime.strptime(dt, '%H:%M:%S.%f')


# Get properties from above
properties = list(df_timedata.keys())
properties.remove('nuts_id')

# Convert strings to time
for property in properties:
    df_timedata[property] = df_timedata[property].apply(apply_time)

gdf_grid = gdf_grid_raw.merge(df_timedata, left_on='NUTS_ID', right_on='nuts_id')
gdf_grid = gdf_grid.drop(columns=['nuts_id'])

df_grouped = gdf_grid.groupby('country').agg(
    sunrise_earliest_current = ('sunrise_earliest_current', 'min'),
    sunrise_latest_current = ('sunrise_latest_current', 'max'),
    sunset_earliest_current = ('sunset_earliest_current', 'min'),
    sunset_latest_current = ('sunset_latest_current', 'max'),

    sunrise_earliest_summer = ('sunrise_earliest_summer', 'min'),
    sunrise_latest_summer = ('sunrise_latest_summer', 'max'),
    sunset_earliest_summer = ('sunset_earliest_summer', 'min'),
    sunset_latest_summer = ('sunset_latest_summer', 'max'),

    sunrise_earliest_winter = ('sunrise_earliest_winter', 'min'),
    sunrise_latest_winter = ('sunrise_latest_winter', 'max'),
    sunset_earliest_winter = ('sunset_earliest_winter', 'min'),
    sunset_latest_winter = ('sunset_latest_winter', 'max')
).reset_index(drop=False)

df_grouped.sort_values(by='sunrise_latest_current', ascending=False, inplace=True)

df_grouped = df_grouped[~df_grouped['country'].isin(['Unknown'])]

# Format for DataWrapper
for property in properties:
    df_grouped[property] = '2025-01-01 ' + df_grouped[property].dt.strftime('%H:%M')

df_grouped.to_clipboard(index=False)

## Alle Zellen exportieren

In [ ]:
# Serialize Timedata
df_timedata = list(map(lambda x: {
    'nuts_id': x['nuts_id'],
    'time': x['sunrise']['current']['latest'],
}, time_data))

df_timedata = pd.DataFrame(df_timedata)

# Convert strings to time
df_timedata['time'] = df_timedata['time'].apply(time_to_time)

def categorize_time_to_2min_numerical(dt):
    return round(dt.hour + dt.minute / 60 + (dt.second / 60) / 60, 2)

df_timedata['time_numerical'] = df_timedata['time'].apply(categorize_time_to_2min_numerical)

gdf_grid = gdf_grid_raw.merge(df_timedata, left_on='NUTS_ID', right_on='nuts_id')
gdf_grid = gdf_grid.drop(columns=['nuts_id'])

gdf_grid['lon'] = round(gdf_grid['geometry'].centroid.x, 2)

# Drop more west than -12
gdf_grid = gdf_grid[gdf_grid['lon'] > -12]

gdf_grid = gdf_grid[['geometry', 'time', 'time_numerical', 'lon', 'timezone']]

gdf_grid.to_file(Path('../export/bubble-data.geojson'))


/var/folders/78/tgbd6qq15fd21h4strxf59mh0000gn/T/ipykernel_19080/77619700.py:20: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_grid['lon'] = round(gdf_grid['geometry'].centroid.x, 2)


In [6]:
gdf_grid

,geometry,time,time_numerical,lon,timezone
0,"MULTIPOLYGON (((15.49746 44.48865, 15.50931 44...",07:34:23.964004,7.57,15.69,Europe/Zagreb
1,"MULTIPOLYGON (((15.90683 44.73362, 15.90479 44...",07:34:31.219161,7.58,15.80,Europe/Sarajevo
2,"MULTIPOLYGON (((26.80834 45.24933, 26.80359 45...",07:51:56.480469,7.87,27.02,Europe/Bucharest
3,"MULTIPOLYGON (((20.49207 38.14231, 20.49106 38...",07:56:54.822770,7.95,20.55,Europe/Athens
4,"MULTIPOLYGON (((-0.13053 51.58157, -0.16194 51...",08:07:31.203637,8.13,-0.34,Europe/London
...,...,...,...,...,...
6833,"MULTIPOLYGON (((-0.95839 41.74478, -0.99244 41...",08:32:08.303280,8.54,-1.18,Europe/Madrid
6834,"MULTIPOLYGON (((21.92462 41.18909, 22.13995 41...",06:57:27.068737,6.96,22.13,Europe/Skopje
6835,"MULTIPOLYGON (((21.91579 40.934, 21.9217 41.10...",07:56:49.363258,7.95,22.13,Europe/Athens
6836,"MULTIPOLYGON (((28.24274 63.33891, 28.3982 63....",09:48:43.492364,9.81,28.47,Europe/Helsinki
